In [1]:
NAME = "Steve Wasiswa"
COLLABORATORS = "Hanna Alemu"

In [2]:
#imports
%matplotlib inline
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics,
                     preprocessing as skpre)


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
# load adult training data and test data
adult_train_df = pd.read_csv("train.csv")
                                
display(adult_train_df.head())
#print(adult_train_df[['workclass']])
# additional observations: 
# education is redundant with education-num, therefore education-num should be used as feature
# id, race, native-country and fnlwght should not be used as features
# sex/hours-per-week should be used as feature since they can influence the income
# age should be used since it has a correlation with income according to data
# race/native-country should not be used since they don't really influence the income
# relationship and marital-status seem redundant; therefore, one should be used which is marital-status
# occupation should be used as feature since it has a huge influence on the income
# workclass,capital-gain and capital-loss should be used as features

# let's focus on only features that seem most useful for now
features = ['age','sex','hours-per-week','education-num','occupation','workclass','capital-gain','capital-loss','marital-status']

# Separate target from features
adult_train_df_ftrs = adult_train_df.drop(['class'], axis=1)
adult_train_df_ftrs = adult_train_df_ftrs[features]
adult_train_tgt = adult_train_df['class']

# Using manual or hardcoded data to clean data
# Convert sex to 0/1 category
adult_train_df_ftrs['sex'].replace(['Male','Female'],[0,1],inplace=True)

adult_train_df_ftrs['occupation'].replace(['Adm-clerical','Exec-managerial','Handlers-cleaners','Prof-specialty',
                                     'Other-service','Sales','Craft-repair','Transport-moving','Farming-fishing',
                                     'Machine-op-inspct','Tech-support','Protective-serv','Other-service',
                                      'Armed-Forces','Priv-house-serv'],[0,1,2,3,4,5,6,7,10,11,12,13,14,15,16],inplace=True)

adult_train_df_ftrs['workclass'].replace(['State-gov','Federal-gov','Local-gov','Self-emp-not-inc','Self-emp-inc','Private',
                                     'Without-pay','Never-worked'],[1,1,1,2,2,0,0,0],inplace=True)

adult_train_df_ftrs['marital-status'].replace(['Never-married','Married-civ-spouse','Divorced','Married-spouse-absent',
                                          'Separated','Widowed','Married-AF-spouse'],[0,1,0,0,1,0,0],inplace=True)

# Replace any question mark with null value
adult_train_df_ftrs = adult_train_df_ftrs.applymap(lambda x: np.NaN if x == '?' else x)

# Replacing all the missing values with the median 
adult_train_df_ftrs = adult_train_df_ftrs.fillna(adult_train_df_ftrs.median())

# Scaling all the training data using RobustScalar scale from sklearn
scaler = skpre.RobustScaler()
scaler.fit(adult_train_df_ftrs)
adult_train_df_ftrs = scaler.transform(adult_train_df_ftrs)


,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [4]:
models_to_try = {'nb': naive_bayes.GaussianNB()}
# add k-NN models with various values of k to models_to_try
for k in range(1,30,2):
    models_to_try[f'{k}-NN'] = neighbors.KNeighborsClassifier(n_neighbors=k)

accuracy_scores = {}
for model_name in models_to_try:
    #loo = skms.LeaveOneOut()
    scores = skms.cross_val_score(models_to_try[model_name],
                                 adult_train_df_ftrs,
                                 adult_train_tgt,
                                 #cv=loo,
                                 cv = 10,
                                 scoring='accuracy')
    mean_accuracy = scores.mean()
    accuracy_scores[model_name] = mean_accuracy
    print(f'{model_name}: {mean_accuracy:.3f}')

best_model_name = max(accuracy_scores,key=accuracy_scores.get)
print(f'\nBest model: {best_model_name} (accuracy = {accuracy_scores[best_model_name]:.3f})')
# set variables for test cell
final_model = models_to_try[best_model_name]

nb: 0.801
1-NN: 0.819
3-NN: 0.843
5-NN: 0.848
7-NN: 0.852
9-NN: 0.854
11-NN: 0.856
13-NN: 0.857
15-NN: 0.858
17-NN: 0.859
19-NN: 0.858
21-NN: 0.858
23-NN: 0.859
25-NN: 0.860
27-NN: 0.860
29-NN: 0.860

Best model: 27-NN (accuracy = 0.860)


In [5]:
# apply final model to test features
# load data
adult_test_df = pd.read_csv("test.csv")
adult_test_df.info() # check for additional null values

adult_test_df_ftrs = adult_test_df[features]
# apply same feature pre-processing steps (using training set for any parameters)
# Using manual or hardcoded data to clean data
# Convert sex to 0/1 category
adult_test_df_ftrs['sex'].replace(['Male','Female'],[0,1],inplace=True)

adult_test_df_ftrs['occupation'].replace(['Adm-clerical','Exec-managerial','Handlers-cleaners','Prof-specialty',
                                     'Other-service','Sales','Craft-repair','Transport-moving','Farming-fishing',
                                     'Machine-op-inspct','Tech-support','Protective-serv','Other-service',
                                      'Armed-Forces','Priv-house-serv'],[0,1,2,3,4,5,6,7,10,11,12,13,14,15,16],inplace=True)

adult_test_df_ftrs['workclass'].replace(['State-gov','Federal-gov','Local-gov','Self-emp-not-inc','Self-emp-inc','Private',
                                     'Without-pay','Never-worked'],[1,1,1,2,2,0,0,0],inplace=True)

adult_test_df_ftrs['marital-status'].replace(['Never-married','Married-civ-spouse','Divorced','Married-spouse-absent',
                                          'Separated','Widowed','Married-AF-spouse'],[0,1,0,0,1,0,0],inplace=True)

# Replace any question mark with null value
adult_test_df_ftrs = adult_test_df_ftrs.applymap(lambda x: np.NaN if x == '?' else x)

# Replacing all the missing values with the median 
adult_test_df_ftrs = adult_test_df_ftrs.fillna(adult_test_df_ftrs.median())

# Scaling all the training data using RobustScalar scale from sklearn
scaler = skpre.RobustScaler()
scaler.fit(adult_test_df_ftrs)
adult_test_df_ftrs = scaler.transform(adult_test_df_ftrs)

model = final_model.fit(adult_train_df_ftrs, adult_train_tgt)
predictions = model.predict(adult_test_df_ftrs)

# create submission dataframe
def writeSubmission(predictions):
    i = 1
    submissionList = []
    counter = 0
    for prediction in predictions:
    #while ( counter < 16281):
        #submissionList.append([str(i), str(predictions[counter])])
        submissionList.append([str(i), str(prediction)])
        i += 1
        
        #counter += 1
                              
    with open('submission.csv', 'w', newline = '') as submission:
        writer = csv.writer(submission)
                              
        writer.writerow(['Id', 'Category'])
        for row in submissionList:
            writer.writerow(row)

writeSubmission(predictions)

# display message
print("Saved predictions to csv file.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              16281 non-null  int64 
 1   age             16281 non-null  int64 
 2   workclass       16281 non-null  object
 3   fnlwgt          16281 non-null  int64 
 4   education       16281 non-null  object
 5   education-num   16281 non-null  int64 
 6   marital-status  16281 non-null  object
 7   occupation      16281 non-null  object
 8   relationship    16281 non-null  object
 9   race            16281 non-null  object
 10  sex             16281 non-null  object
 11  capital-gain    16281 non-null  int64 
 12  capital-loss    16281 non-null  int64 
 13  hours-per-week  16281 non-null  int64 
 14  native-country  16281 non-null  object
dtypes: int64(7), object(8)
memory usage: 1.9+ MB


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Saved predictions to csv file.
